In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras import layers

# 創建時間序列資料
#tran_data = [2, 7, 9, 11, 17, 21, 15, 2, 10, 9, 15, 20, 16, 12, 9, 7, 15, 9, 9, 16]
# 讀取Excel文件中的數據
df = pd.read_excel("data.xlsx", sheet_name="tran_data")

# 將數據轉換為Numpy數組
tran_data = df["Column_Name"].values  # 將 "Column_Name" 替換為包含數據的列名

# 將資料轉換為Numpy數組
tran_data = np.array(tran_data, dtype=float)

# 分割數據為輸入和目標
sequence_length = 10  # 每個輸入序列的長度
X = []
y = []

for i in range(len(tran_data) - sequence_length):
    X.append(tran_data[i:i+sequence_length])
    y.append(tran_data[i+sequence_length])

X = np.array(X)
y = np.array(y)

# 正規化數據（可選）
# X = (X - np.min(X)) / (np.max(X) - np.min(X))
# y = (y - np.min(y)) / (np.max(y) - np.min(y))

# 創建RNN模型
model = keras.Sequential()
model.add(layers.SimpleRNN(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(layers.Dense(1))

# 編譯模型
model.compile(optimizer='adam', loss='mean_squared_error')

# 訓練模型
model.fit(X, y, epochs=100, batch_size=4)

# 使用模型預測下一個值
last_sequence = np.array(tran_data[-sequence_length:]).reshape(1, sequence_length, 1)
predicted_value = model.predict(last_sequence)
print("預測的下一個值為:", predicted_value[0][0])


# 保存模型權重
model.save("time_series_model.h5")

In [ ]:
## 持續訓練該模型
# 載入模型權重
model.load_weights("time_series_model.h5")

# 讀取Excel文件中的數據
df = pd.read_excel("data.xlsx", sheet_name="tran_data")

# 將數據轉換為Numpy數組
tran_data = df["Column_Name"].values  # 將 "Column_Name" 替換為包含數據的列名

# 將資料轉換為Numpy數組
tran_data = np.array(tran_data, dtype=float)

# 分割數據為輸入和目標
sequence_length = 10  # 每個輸入序列的長度
X = []
y = []

for i in range(len(tran_data) - sequence_length):
    X.append(tran_data[i:i+sequence_length])
    y.append(tran_data[i+sequence_length])

X = np.array(X)
y = np.array(y)

# 繼續訓練模型
model.fit(X, y, epochs=200, batch_size=4)

In [ ]:
# 載入模型權重
model.load_weights("time_series_model.h5")

input_data = np.array([17,11,9,7,12,8,14,9])

# 將 input_data 補齊為長度為10的序列
if len(input_data) < 10:
    input_data = np.concatenate(([0] * (10 - len(input_data)), input_data), axis=0)

# 將 input_data 轉換為二維格式
input_data = input_data.reshape(1, 10, 1)

# 使用模型預測下一筆資料
predicted_value = model.predict(input_data)

print("預測的下一筆資料為:", predicted_value[0][0])

In [ ]:
##LSTM
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 創建時間序列資料
#tran_data = [2, 7, 9, 11, 17, 21, 15, 2, 10, 9, 15, 20, 16, 12, 9, 7, 15, 9, 9, 16]
# 讀取Excel文件中的數據
df = pd.read_excel("20231031_tran_data.xlsx", sheet_name="tran_data")

# 將數據轉換為Numpy數組
tran_data = df["Column_Name"].values  # 將 "Column_Name" 替換為包含數據的列名

# 將資料轉換為Numpy數組
tran_data = np.array(tran_data, dtype=float)

# 分割數據為輸入和目標
sequence_length = 10  # 每個輸入序列的長度
X = []
y = []

for i in range(len(tran_data) - sequence_length):
    X.append(tran_data[i:i+sequence_length])
    y.append(tran_data[i+sequence_length])

X = np.array(X)
y = np.array(y)

# 正規化數據（可選）
# X = (X - np.min(X)) / (np.max(X) - np.min(X))
# y = (y - np.min(y)) / (np.max(y) - np.min(y))

# 創建LSTM模型
model = keras.Sequential()
model.add(layers.LSTM(50, activation='relu', input_shape=(sequence_length, 1), return_sequences=True))
model.add(layers.LSTM(25, activation='relu', input_shape=(sequence_length, 1),return_sequences=False))
model.add(Dropout(0.1))
model.add(layers.Dense(1))

# 編譯模型
model.compile(optimizer='adam', loss='mean_squared_error')

# 訓練模型
history = model.fit(X, y, epochs=200, batch_size=16)

# 繪製訓練過程的損失函數收斂曲線
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()

# 使用模型預測下一個值
last_sequence = np.array(tran_data[-sequence_length:]).reshape(1, sequence_length, 1)
predicted_value = model.predict(last_sequence)
print("預測的下一個值為:", predicted_value[0][0])

# 保存模型權重
model.save("time_series_LSTM_model.h5")

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# 定義建立模型的函數
def create_model(units=50, learning_rate=0.001, batch_size=4, epochs=100):
    # model = keras.Sequential()
    # model.add(layers.LSTM(units, activation='relu', input_shape=(sequence_length, 1)))
    # model.add(layers.Dense(1))
    # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')
    # 讀取Excel文件中的數據
    df = pd.read_excel("20231031_tran_data.xlsx", sheet_name="tran_data")
    # 將數據轉換為Numpy數組
    tran_data = df["Column_Name"].values  # 將 "Column_Name" 替換為包含數據的列名

    # 將資料轉換為Numpy數組
    tran_data = np.array(tran_data, dtype=float)

    # 分割數據為輸入和目標
    sequence_length = 10  # 每個輸入序列的長度
    X = []
    y = []

    for i in range(len(tran_data) - sequence_length):
        X.append(tran_data[i:i+sequence_length])
        y.append(tran_data[i+sequence_length])

    X = np.array(X)
    y = np.array(y)

    # 正規化數據（可選）
    # X = (X - np.min(X)) / (np.max(X) - np.min(X))
    # y = (y - np.min(y)) / (np.max(y) - np.min(y))

    # 創建LSTM模型
    model = keras.Sequential()
    model.add(layers.LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
    model.add(layers.Dense(1))

    # 編譯模型
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 使用KerasRegressor包裝模型，以便於GridSearchCV使用
model = KerasRegressor(build_fn=create_model, verbose=0)

# 定義要調整的超參數範圍
param_grid = {
    'units': [50, 100, 150,200],
    'learning_rate': [0.001, 0.01, 0.1,0.2,0.02,0.5,0.05],
    'batch_size': [4, 8, 16, 20],
    'epochs': [50, 100, 200, 150]
}

# 使用網格搜索來找到最佳超參數組合
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X, y)

# 輸出最佳超參數組合和最佳評分
print("最佳超參數組合: ", grid_result.best_params_)
print("最佳評分: ", -grid_result.best_score_)

In [1]:
##LSTM 預測二元分類
import numpy as np
import pandas as pd

# tran_data = ['S', 'S', 'B', 'S', 'B', 'S', 'B', 'B', 'B', 'S', 'S', 'B', 'S', 'B', 'S', 'B', 'B', 'B', 'S']
# tran_data_numeric = [0 if x == 'S' else 1 for x in tran_data]

# 讀取Excel文件中的數據
df = pd.read_excel("20231101_lucky_number.xlsx", sheet_name="tran_data")

# 將數據轉換為Numpy數組
tran_data = df["Column_Name"].values  # 將 "Column_Name" 替換為包含數據的列名

tran_data_numeric = [0 if x == 'S' else 1 for x in tran_data]



In [2]:
sequence_length = 10
X = []
y = []

for i in range(len(tran_data_numeric) - sequence_length):
    X.append(tran_data_numeric[i:i+sequence_length])
    y.append(tran_data_numeric[i+sequence_length])

X = np.array(X)
y = np.array(y)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np  # 確保已導入 NumPy
import matplotlib.pyplot as plt

model = keras.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(sequence_length, 1),return_sequences=True))
model.add(layers.LSTM(80, activation='relu', input_shape=(sequence_length, 1),return_sequences=True))
model.add(layers.LSTM(50, activation='relu', input_shape=(sequence_length, 1),return_sequences=False))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history =model.fit(X, y, epochs=460, batch_size=16)
print("history",history)
# 繪製訓練過程的損失函數收斂曲線
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()


# 保存模型權重
model.save("lucky_number_LSTM_model.h5")

In [ ]:
# 使用模型預測下一筆資料
# 從 HDF5 檔案中載入模型
model = keras.models.load_model('lucky_number_LSTM_model.h5')
last_sequence = np.array(tran_data_numeric[-sequence_length:]).reshape(1, sequence_length, 1)
predicted_value = model.predict(last_sequence)

predicted_class = "B" if predicted_value[0][0] > 0.5 else "S"
print("預測的下一筆資料為:", predicted_class)

In [ ]:
# 初始化一個空的預測序列
predicted_sequence = []

# 進行 5 次預測
for i in range(10):
    # 最後 sequence_length 筆資料的預測
    last_sequence = np.array(tran_data_numeric[-sequence_length:]).reshape(1, sequence_length, 1)
    predicted_value = model.predict(last_sequence)

    predicted_class = "B" if predicted_value[0][0] > 0.5 else "S"
    
    # 添加預測值到預測序列
    predicted_sequence.append(predicted_class)
    
    # 更新 tran_data_numeric，以便下一次預測
    tran_data_numeric.append(0 if predicted_class == 'S' else 1)

print("預測的下一 5 筆資料為:", predicted_sequence)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras import layers

In [ ]:
# 使用模型预测接下来的 1 筆資料
predicted_sequence = []
test_data_numeric = ['B','B','S','S','S','S','S','S','B','S']

# 從 HDF5 檔案中載入模型
model = keras.models.load_model('lucky_number_LSTM_model.h5')
# 获取过去的 10 个时间步数据
past_data = test_data_numeric

# 将过去的数据转换为模型期望的输入格式
input_sequence = np.array([0 if x == 'S' else 1 for x in past_data]).reshape(1, 10, 1)
print('input_sequence',input_sequence)
# 使用模型进行预测
predicted_value = model.predict(input_sequence)

predicted_class = "B" if predicted_value[0][0] > 0.5 else "S"

print("預測的下一筆資料為:", predicted_class)

In [3]:
# 初始化一個空的預測序列
# 使用模型预测接下来的 5 筆資料
sequence_length = 10
predicted_sequence = []
test_data_numeric = ['B','B','S','S','S','S','S','S','B','S']
# 從 HDF5 檔案中載入模型
model = keras.models.load_model('lucky_number_LSTM_model.h5')
past_data = test_data_numeric
# 進行 10 次預測
for i in range(10):
    # 最後 sequence_length 筆資料的預測
    input_sequence = np.array([0 if x == 'S' else 1 for x in past_data[-sequence_length:]]).reshape(1, sequence_length, 1)
    
    predicted_value = model.predict(input_sequence)

    predicted_class = "B" if predicted_value[0][0] > 0.5 else "S"
    
    # 添加預測值到預測序列
    predicted_sequence.append(predicted_class)
    
    # 更新 tran_data_numeric，以便下一次預測
    test_data_numeric.append(0 if predicted_class == 'S' else 1)

print("預測的下一 5 筆資料為:", predicted_sequence)

1/1 [==============================] - 0s 26ms/step
預測的下一 5 筆資料為: ['S', 'S', 'B', 'S', 'B', 'S', 'S', 'S', 'S', 'S']
